In [2]:
import pandas as pd # Para los dataframes y manipular los csv
import numpy as np # Para los arrays y operaciones matemáticas
import pulp as pl # Librería de programación lineal
import matplotlib.pyplot as plt # Librería para graficar (capaz la uso para visualizar al final, la cargo por las dudas)

# Cargo los datasets que manipulé en Limpieza-Preliminar.ipynb
depositos = pd.read_csv('Archivos Intermedios/filtered_depositos.csv')
mayoristas = pd.read_csv('Archivos Intermedios/filtered_mayoristas.csv')
productos = pd.read_csv('Archivos Intermedios/filtered_productos.csv')

In [ ]:
# Defino unos índices (listas de los contenidos de las columnas) para facilitar la operación del código

mayoristas_list = mayoristas['mayorista'].tolist()  # M1, M2, M3
depositos_list = depositos['deposito'].tolist()     # D1_DEP, D2_DEP
pdv_list = [f'pdv_{i}' for i in range(1, 6)]        # pdv_1, ..., pdv_5
productos_list = productos['producto_id'].tolist()

# Armo un diccionario para los productos (lo voy a tener que usar en las funciones más adelante)
volumen_prod = {p: productos[productos['producto_id'] == p].iloc[0]['volumen_m3'] for p in productos_list}

# Defino la función resolver_capacidad; para poder resolver con un aumento de capacidad específico. (me sirve para el punto 3)

def resolver_capacidad(d1_capacity_increase=0):
    
    # Tiene que devolver el valor óptimo del funcional minimizando costos
    
    # Primero armo el modelo de PuLP
    model = pl.LpProblem("Distribution_Optimization", pl.LpMinimize)
    
    # Acá es donde cargaría mis aumentos de la capacidad (del 3)
    capacidad_depositos = {d: depositos[depositos['deposito'] == d].iloc[0]['capacidad_m3'] for d in depositos_list}
    capacidad_depositos['D1_DEP'] += d1_capacity_increase
    
    # Armo mis variables de decisión:
    # 1. X_m_p_d: cantidad de producto (p) del mayorista (m) al deposito (d)
    X_m_p_d = {(m, p, d): pl.LpVariable(f"X_{m}_{p}_{d}", lowBound=0, cat='Integer')
               for m in mayoristas_list
               for p in productos_list
               for d in depositos_list}
    
    # 2. Y_m_p_v: Cantidad de producto (p) del mayorista (m) directo al punto de venta (v)
    Y_m_p_v = {(m, p, v): pl.LpVariable(f"Y_{m}_{p}_{v}", lowBound=0, cat='Integer')
               for m in mayoristas_list
               for p in productos_list
               for v in pdv_list}
    
    # 3. Z_d_p_v: Cantidad de producto (p) del deposito (d) al punto de venta (v)
    Z_d_p_v = {(d, p, v): pl.LpVariable(f"Z_{d}_{p}_{v}", lowBound=0, cat='Integer')
               for d in depositos_list
               for p in productos_list
               for v in pdv_list}
    
    # Armo diccionarios para los costos de compra para tenerlo a mano
    # Costo de comprar producto (p) al mayorista (m)
    buy_cost = {(m, p): 0 for m in mayoristas_list for p in productos_list}
    for p in productos_list:
        p_row = productos[productos['producto_id'] == p].iloc[0]
        buy_cost[('M1', p)] = p_row['costo_M1_usd']
        buy_cost[('M2', p)] = p_row['costo_M2_usd']
        buy_cost[('M3', p)] = p_row['costo_M3_usd']
    
    # Costo de transportar del mayorista (m) al deposito (d) por m³
    transport_m_to_d = {(m, d): 0 for m in mayoristas_list for d in depositos_list}
    for m in mayoristas_list:
        m_row = mayoristas[mayoristas['mayorista'] == m].iloc[0]
        for d in depositos_list:
            transport_m_to_d[(m, d)] = m_row[d]
    
    # Costo de transportar del mayorista (m) al punto de venta (v) por m³
    transport_m_to_v = {(m, v): 0 for m in mayoristas_list for v in pdv_list}
    for m in mayoristas_list:
        m_row = mayoristas[mayoristas['mayorista'] == m].iloc[0]
        for v in pdv_list:
            transport_m_to_v[(m, v)] = m_row[v]
    
    # Costo de transportar del deposito (d) al punto de venta (v) por m³ 
    transport_d_to_v = {(d, v): 0 for d in depositos_list for v in pdv_list}
    for d in depositos_list:
        d_row = depositos[depositos['deposito'] == d].iloc[0]
        for v in pdv_list:
            transport_d_to_v[(d, v)] = d_row[v] / 10  # USD/(10m³) -> USD/m³
    
    # Diccionario de demanda para productos en cada PDV
    demand = {(p, v): 0 for p in productos_list for v in pdv_list}
    for p in productos_list:
        p_row = productos[productos['producto_id'] == p].iloc[0]
        for v in pdv_list:
            demand[(p, v)] = p_row[v]
    
    # Función Objetivo: Minimizar costos tal que:
    # Costo = costo de compra + de transporte
    func_objetivo = []
    
    # 1. Costo de compra a mayoristas
    for m in mayoristas_list:
        for p in productos_list:
            # Suma de productoss que van a depositos para cada mayorista
            deposit_amount = sum(X_m_p_d[(m, p, d)] for d in depositos_list)
            # Suma de productoss que van a Puntos de Venta para cada mayorista
            direct_amount = sum(Y_m_p_v[(m, p, v)] for v in pdv_list)
            # Costo total
            func_objetivo.append(buy_cost[(m, p)] * (deposit_amount + direct_amount))
    
    # 2. Costo de transporte de mayorista a deposito
    for m in mayoristas_list:
        for p in productos_list:
            for d in depositos_list:
                vol = volumen_prod[p]
                func_objetivo.append(transport_m_to_d[(m, d)] * vol * X_m_p_d[(m, p, d)])
    
    # 3. Costo de transporte de mayorista directo a los PDV
    for m in mayoristas_list:
        for p in productos_list:
            for v in pdv_list:
                vol = volumen_prod[p]
                func_objetivo.append(transport_m_to_v[(m, v)] * vol * Y_m_p_v[(m, p, v)])
    
    # 4. Costo de transporte de depositos a PDVs
    for d in depositos_list:
        for p in productos_list:
            for v in pdv_list:
                vol = volumen_prod[p]
                func_objetivo.append(transport_d_to_v[(d, v)] * vol * Z_d_p_v[(d, p, v)])
    
    # Ahora sí, planteo el modelo teniendo en cuenta la función objetivo
    model += pl.lpSum(func_objetivo)
    
    # Restricciones
    
    # 1. Satisfacción de la demanda por cada producto en cada PDV
    for p in productos_list:
        for v in pdv_list:
            model += (
                sum(Y_m_p_v[(m, p, v)] for m in mayoristas_list) +
                sum(Z_d_p_v[(d, p, v)] for d in depositos_list)
                == demand[(p, v)],
                f"Demand_Satisfaction_{p}_{v}"
            )
    
    # 2. Esto está para asegurarse que lo de los depositos no se quede estancado ahí (what comes in must go out)
    for d in depositos_list:
        for p in productos_list:
            model += (
                sum(X_m_p_d[(m, p, d)] for m in mayoristas_list) ==
                sum(Z_d_p_v[(d, p, v)] for v in pdv_list),
                f"Flow_Conservation_{d}_{p}"
            )
    
    # 3. Acá se ponen los límites de volumen por deposito
    for d in depositos_list:
        model += (
            sum(volumen_prod[p] * X_m_p_d[(m, p, d)] for m in mayoristas_list for p in productos_list)
            <= capacidad_depositos[d],
            f"Capacity_{d}"
        )
    
    # Acá hago que se resuelva el modelo
    solver = pl.PULP_CBC_CMD(msg=False)
    model.solve(solver)
    
    # Hago que retorne los datos del modelo así los puedo usar para sacar los resultados
    return pl.value(model.objective), model

# Acá busca el valor óptimo asumiendo que la capacidad extra sea 0
base_objective, base_model = resolver_capacidad(0)
print(f"Base optimal cost: ${base_objective:.2f}")

# Hago este código para que me genere los csv de las órdenes de compra (es medio un chino pero sirve para automatizarlo por si me sirve para otro tp)
def generate_buy_orders_csv(model, mayoristas_list, productos_list, depositos_list, pdv_list):
    for m in mayoristas_list:
        columnas = []
        for p in productos_list:
            # cantidad de producto (p) del mayorista (m) al deposito (d)
            d1_cant = sum(
                model.variablesDict()[f"X_{m}_{p}_D1_DEP"].value()
                for d in depositos_list if f"X_{m}_{p}_D1_DEP" in model.variablesDict()
            )
            d2_cant = sum(
                model.variablesDict()[f"X_{m}_{p}_D2_DEP"].value()
                for d in depositos_list if f"X_{m}_{p}_D2_DEP" in model.variablesDict()
            )
            
            # cantidad de producto (p) del mayorista (m) directo al punto de venta (v)
            pdv_cant = [
                model.variablesDict()[f"Y_{m}_{p}_{v}"].value()
                if f"Y_{m}_{p}_{v}" in model.variablesDict() else 0
                for v in pdv_list
            ]
            
            # Saco la cantidad total 
            cant_total = d1_cant + d2_cant + sum(pdv_cant)
            
            # Append the row
            columnas.append({
                "producto_id": p,
                "cantidad_total": cant_total,
                "D1_DEP": d1_cant,
                "D2_DEP": d2_cant,
                **{v: pdv_cant[i] for i, v in enumerate(pdv_list)}
            })
        
        # Ahora sí armo el dataframe y 
        df = pd.DataFrame(columnas)
        df.to_csv(f"Resultados-Finales/Orden-de-Compra-{m}.csv", index=False)
        print(f"impreso el {m}.")

# Llamo la función para que me genere los csv
generate_buy_orders_csv(
    base_model,
    mayoristas_list,
    productos_list,
    depositos_list,
    pdv_list
)

# Trato de sacar el costo marginal (todo este tramo con los comentarios en inglés se lo debo a Claude)
costo_marginal = None
for name, constraint in base_model.constraints.items():
    if name == "Capacity_D1_DEP":
        try:
            costo_marginal = -constraint.pi  # Shadow price is the negative of the dual variable
            print(f"Shadow price for D1_DEP capacity: ${costo_marginal:.4f} per cubic meter")
        except:
            print("Could not get shadow price directly from the constraint, will calculate it by finite difference")

# Calculate shadow price by finite difference if not available directly
if costo_marginal is None:
    small_increase = 1
    new_objective, _ = resolver_capacidad(small_increase)
    costo_marginal = (base_objective - new_objective) / small_increase
    print(f"Shadow price for D1_DEP capacity (calculated by finite difference): ${costo_marginal:.4f} per cubic meter")

# Acá es dónde pruebo con distintos aumentos de capacidad para ver todos los datos 
increases = [1, 1000, 100000]
results = []

for increase in increases:
    new_objective, _ = resolver_capacidad(increase)
    cost_savings = base_objective - new_objective
    average_value_per_unit = cost_savings / increase
    results.append({
        'Incremento de D1 (m³)': increase,
        'Costo minimo ($)': new_objective,
        'Ahorro ($)': cost_savings,
        'Average Value per m³': average_value_per_unit
    })

# Imprimo
results_df = pd.DataFrame(results)
print("\Tabla de capacidades (y el valor medio ya que estamos):")
print(results_df)

# Calculate the capacity utilization in the base case
print("\nUso de los depositos:")
capacidad_depositos = {d: depositos[depositos['deposito'] == d].iloc[0]['capacidad_m3'] for d in depositos_list}
for d in depositos_list:
    # busco todos los d
    used_capacity = 0
    for m in mayoristas_list:
        for p in productos_list:
            var_name = f"X_{m}_{p}_{d}"
            if var_name in base_model.variablesDict():
                var_value = base_model.variablesDict()[var_name].value()
                if var_value is not None:
                    used_capacity += volumen_prod[p] * var_value
    
    print(f"{d} utilization: {used_capacity:.2f} m³ out of {capacidad_depositos[d]} m³ ({100*used_capacity/capacidad_depositos[d]:.2f}%)")


<>:240: SyntaxWarning: invalid escape sequence '\T'
<>:240: SyntaxWarning: invalid escape sequence '\T'
/tmp/ipykernel_14936/221533164.py:240: SyntaxWarning: invalid escape sequence '\T'
  print("\Tabla de capacidades (y el valor medio ya que estamos):")


Base optimal cost: $1619871.01
impreso el M1.
impreso el M2.
impreso el M3.
Shadow price for D1_DEP capacity: $-0.0000 per cubic meter
\Tabla de capacidades (y el valor medio ya que estamos):
   Incremento de D1 (m³)  Costo minimo ($)   Ahorro ($)  Average Value per m³
0                      1      1.619862e+06     8.762000              8.762000
1                   1000      1.613477e+06  6393.526256              6.393526
2                 100000      1.613477e+06  6393.526256              0.063935

Uso de los depositos:
D1_DEP utilization: 1500.00 m³ out of 1500 m³ (100.00%)
D2_DEP utilization: 1300.00 m³ out of 1300 m³ (100.00%)

Maximum valuable capacity increase:


KeyError: 'Capacity Increase (m³)'

Verificación de que tengan sentido los resultados:

In [ ]:
M3 = pd.read_csv('Resultados-Claude/M3_buy_orders.csv')

In [ ]:
M3["cantidad_total"].sum()